<a href="https://colab.research.google.com/github/edwige3/Lab_2_Introduction-to-wordvectors/blob/main/Raissa_Edwige__intro_to_wordvectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 2: Introduction to wordvectors </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [57]:
import io, sys
import numpy as np

In [58]:
def load_vectors(filename):
    fin = io.open(filename, 'r', encoding='utf-8', newline='\n')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.asarray([float(x) for x in tokens[1:]])
    return data

In [60]:
# Loading word vectors

print('')
print(' ** Word vectors ** ')
print('')

'''
word_vectors is a dictionary that maps words to their numerical word vector
[word (string)] = [np-array] 
'''
word_vectors = load_vectors('wiki.en.vec')

tree_vector = word_vectors['tree']
print(type(tree_vector), len(tree_vector))


 ** Word vectors ** 

<class 'numpy.ndarray'> 300


In [61]:
## This function computes the cosine similarity between vectors u and v

def cosine(u, v):
    '''
    Parameters:
    u : 1-D numpy array
    v : 1-D numpy array 
    
    Returns:
    cos (float) : value of the cosine similairy between vectors u, v 
    '''
    ## FILL CODE
    # Compute the dot product between u and v (≈1 line) , or the scalar , we can also used  (u transpose * v)
    dot = np.dot(u, v) 
    # Compute the L2 norm of u (≈1 line)
    norm_u = np.sqrt(np.sum(u**2))
    # Compute the L2 norm of v (≈1 line)
    norm_v = np.sqrt(np.sum(v**2))
    # Compute the cosine similarity defined by formula (1) (≈1 line)
    cos = dot / np.dot(norm_u, norm_v)
    return cos 


In [62]:
# compute similarity between words

print('similarity(apple, apples) = %.3f' %
      cosine(word_vectors['apple'], word_vectors['apples']))
print('similarity(apple, banana) = %.3f' %
      cosine(word_vectors['apple'], word_vectors['banana']))
print('similarity(apple, tiger) = %.3f' %
      cosine(word_vectors['apple'], word_vectors['tiger']))

similarity(apple, apples) = 0.637
similarity(apple, banana) = 0.431
similarity(apple, tiger) = 0.212


In [71]:
def nearest_neighbor(x, word_vectors, exclude_words=[]):
    '''
    Parameters:
    x (string): word to find its nearest neighbour 
    word_vectors (Python dict): {word (string): np-array of word vector}
    exclude_words (list of strings): words to be excluded from the search
    
    Returns:
    best_word (string) : the word whose word vector is the nearest neighbour 
    to the word vector of x
    '''
    best_score = -1.0
    best_word = None

    ## FILL CODE
    
    key_list = list(word_vectors.keys())
    liscos=[] #compute the similarity(list of sim)
    for word in  word_vectors:
      if word not in exclude_words:
        liscos.append(cosine(x,word_vectors[word]))

    nearn=np.argmax(liscos)
    best_word=key_list[nearn]
            
    return best_word

In [72]:
print('')
print('The nearest neighbor of cat is: ' +
      nearest_neighbor(word_vectors['cat'], word_vectors, exclude_words = ['cat', 'cats']))


The nearest neighbor of cat is: dog


#### Hint (using python priorty queues with the heapq datastructure): 
if you don't want to store all the words and scores you can use the priortiy queue and only store the best K element so far. 

In [73]:
## This function return the words corresponding to the
## K nearest neighbors of vector x.
## You can use the functions heappush and heappop.

def knn(x, vectors, k):
    '''
    Parameters:
    x (string): word to find its nearest neighbour 
    word_vectors (Python dict): {word (string): np-array of word vector}
    k (int): number of nearest neighbours to be found
    
    Returns: 
    k_nearest_neighbors (list of tuples): [(score, word), (score, word), ....]
    '''

    k_nearest_neighbors =[]
    ## FILL CODE
    key_list = list(vectors.keys())
    liscos=[]#compute the similarity(list of sim)
    for word in  vectors:
      liscos.append(cosine(x,vectors[word]))

    liscosort=np.argsort(liscos)
    largest_indices = liscosort[::-1][:k]
    for i in largest_indices:
      k_nearest_neighbors.append((liscos[i],key_list[i]))
   
    return k_nearest_neighbors

In [74]:
knn_cat = knn(word_vectors['cat'], word_vectors, 5)
print('')
print('cat')
print('--------------')
for score, word in knn(word_vectors['cat'], word_vectors, 5):
    print (word + '\t%.3f' % score)



cat
--------------
cat	1.000
cats	0.732
dog	0.638
pet	0.573
rabbit	0.549


#### Hint: 
To find the analogies, we find the nearest neighbour associated with the wordvector d
$$ d = \frac{c}{\Vert {c} \Vert} + \frac{b}{\Vert {b} \Vert} - \frac{a}{\Vert {a} \Vert}$$


In [115]:
## This function return the words d, such that a:b and c:d
## verifies the same relation

def analogy(a, b, c, word_vectors):
    '''
    Parameters:
    a (string): word a
    b (string): word b
    c (string): word c
    word_vectors (Python dict): {word (string): np-array of word vector}
    
    Returnrs: 
    the word d (string) associated with c such that c:d is similar to a:b 
    
    '''
    ## FILL CODE
    vecs_a= word_vectors[a]
    vecs_b= word_vectors[b]
    vecs_c= word_vectors[c]


    norm_a = np.linalg.norm(vecs_a)
    norm_c = np.linalg.norm(vecs_c)
    norm_b = np.linalg.norm(vecs_b)

    d =(vecs_c/norm_c)+(vecs_b/ norm_b ) - (vecs_a/norm_a)

    #return x

    return nearest_neighbor(d, word_vectors)

In [116]:
# Word analogies

print('')
print('france - paris + rome = ' + analogy('paris', 'france', 'rome', word_vectors))


france - paris + rome = italy


## A word about biases in word vectors

In [112]:
## A word about biases in word vectors:

print('')
print('similarity(genius, man) = %.3f' %
      cosine(word_vectors['man'], word_vectors['genius']))
print('similarity(genius, woman) = %.3f' %
      cosine(word_vectors['woman'], word_vectors['genius']))


similarity(genius, man) = 0.445
similarity(genius, woman) = 0.325


In [145]:
## Compute the association strength between:
##   - a word w
##   - two sets of attributes A and B

def association_strength(w, A, B, vectors):
    '''
    Parameters:
    w (string): word w
    A (list of strings): The words belonging to set A
    B (list of strings): The words belonging to set B
    vectors (Python dict): {word (string): np-array of word vector}
    
    Returnrs: 
    strength (float): the value of the association strength 
    '''
    
    # strength = 0.0
    part_a = 0.0
    part_b = 0.0 
    
    ## FILL CODE
    card_A = 1/len(A)
    card_B = 1/len(B)
    for i in A:
      print(i)
      sum_A = cosine(vectors[w],vectors[i])
      part_a +=sum_A
    for j in B:
      sum_B = cosine(vectors[w],vectors[j])
      part_b +=sum_B
    
    strength = card_A * part_a - card_A * part_b

    return strength

In [149]:
association_strength('home', male, female, word_vectors)

john
paul
mike
kevin
steve
greg
jeff
bill


-0.027292590003828443

In [154]:
## Perform the word embedding association test between:
##   - two sets of words X and Y
##   - two sets of attributes A and B

def weat(X, Y, A, B, vectors):
    '''
    Parameters:
    X (list of strings): The words belonging to set X
    Y (list of strings): The words belonging to set Y
    A (list of strings): The words belonging to set A
    B (list of strings): The words belonging to set B
    vectors (Python dict): {word (string): np-array of word vector}
    
    Returns: 
    score (float): the value of the group association strength  
    '''
    
    score = 0.0
    strength_Part_A =0.0
    strength_Part_B =0.0
    ## FILL CODE

    for x in X :
      Part_1 = association_strength(x, A, B, vectors)
      print(Part_1)
      strength_Part_A += Part_1

    for y in Y:
      Part_2 = association_strength(y, A, B, vectors)
      strength_Part_B += Part_2


    score = strength_Part_A - strength_Part_B
    return score

In [144]:
word_vectors.keys()

dict_keys([',', '.', 'the', '</s>', 'of', '-', 'in', 'and', "'", ')', '(', 'to', 'a', 'is', 'was', 'on', 's', 'for', 'as', 'by', 'that', 'it', 'with', 'from', 'at', 'he', 'this', 'be', 'i', 'an', 'utc', 'his', 'not', '–', 'are', 'or', 'talk', 'which', 'also', 'has', 'were', 'but', 'have', '#', 'one', 'rd', 'new', 'first', 'page', 'no', 'you', 'they', 'had', 'article', 't', 'who', '?', 'all', 'their', 'there', 'been', 'made', 'its', 'people', 'may', 'after', '%', 'other', 'should', 'two', 'score', 'her', 'can', 'would', 'more', 'if', 'she', 'about', 'when', 'time', 'team', 'american', 'such', 'th', 'do', 'discussion', 'links', 'only', 'some', 'up', 'see', 'united', 'years', 'into', '/', 'school', 'so', 'world', 'university', 'during', 'out', 'state', 'states', 'national', 'wikipedia', 'year', 'most', 'city', 'over', 'used', 'then', 'd', 'than', 'county', 'external', 'm', 'where', 'will', 'de', 'what', 'delete', 'any', 'these', 'january', 'march', 'august', 'july', 'being', 'film', 'him'

In [155]:
## Replicate one of the experiments from:
##
## Semantics derived automatically from language corpora contain human-like biases
## Caliskan, Bryson, Narayanan (2017)

career = ['executive', 'management', 'professional', 'corporation', 
          'salary', 'office', 'business', 'career']
family = ['home', 'parents', 'children', 'family',
          'cousins', 'marriage', 'wedding', 'relatives']
male = ['john', 'paul', 'mike', 'kevin', 'steve', 'greg', 'jeff', 'bill']
female = ['amy', 'joan', 'lisa', 'sarah', 'diana', 'kate', 'ann', 'donna']

print('')
print('Word embedding association test: %.3f' %
      weat(career, family, male, female, word_vectors))


john
paul
mike
kevin
steve
greg
jeff
bill
0.01713961933247468
john
paul
mike
kevin
steve
greg
jeff
bill
0.008555922629500418
john
paul
mike
kevin
steve
greg
jeff
bill
0.07517039362220646
john
paul
mike
kevin
steve
greg
jeff
bill
0.020974561160362143
john
paul
mike
kevin
steve
greg
jeff
bill
0.03826680984950162
john
paul
mike
kevin
steve
greg
jeff
bill
-0.009149594561707386
john
paul
mike
kevin
steve
greg
jeff
bill
-0.002031471271222135
john
paul
mike
kevin
steve
greg
jeff
bill
0.05856239715589301
john
paul
mike
kevin
steve
greg
jeff
bill
john
paul
mike
kevin
steve
greg
jeff
bill
john
paul
mike
kevin
steve
greg
jeff
bill
john
paul
mike
kevin
steve
greg
jeff
bill
john
paul
mike
kevin
steve
greg
jeff
bill
john
paul
mike
kevin
steve
greg
jeff
bill
john
paul
mike
kevin
steve
greg
jeff
bill
john
paul
mike
kevin
steve
greg
jeff
bill
Word embedding association test: 0.847


## Word translation using word vectors

In the following, we will use word vectors in English and French to translate words from English to French. The idea is to learn a linear function that maps English word vectors to their correponding French word vectors. To learn this linear mapping, we will use a small bilingual lexicon, that contains pairs of words in English and French that are translations of each other.

The following function will load the small English-French bilingual lexicon:

In [138]:
def load_lexicon(filename):
    '''
    Parameters:
    filename(string): the path of the lexicon
    
    Returns:
    data(list of pairs of string): the bilingual lexicon
    '''
    fin = io.open(filename, 'r', encoding='utf-8', newline='\n')
    data = []
    for line in fin:
        a, b = line.rstrip().split(' ')
        data.append((a, b))
    return data

In [139]:
word_vectors_en = load_vectors('wiki.en.vec')
word_vectors_fr = load_vectors('wiki.fr.vec')
lexicon = load_lexicon("lexicon-en-fr.txt")
print(lexicon[:5])

[('the', 'le'), ('the', 'les'), ('the', 'la'), ('and', 'et'), ('was', 'fut')]


In [140]:
# We split the lexicon into a train and validation set
train = lexicon[:5000]
valid = lexicon[5000:5100]

The following function will learn the mapping from English to French. The idea is to build two matrices $X_{\text{en}}$ and $X_{\text{fr}}$, and to find a mapping $M$ that minimizes $||X_{\text{en}} W - X_{\text{fr}} ||_2$. In numpy, this mapping can be obtained by using the `numpy.linalg.lstsq` function.

In [156]:
def align(word_vectors_en, word_vectors_fr, lexicon):
    '''
    Parameters:
    word_vectors_en(dict: string -> np.array): English word vectors
    word_vectors_en(dict: string -> np.array): French word vectors
    lexicon(list of pairs of string): bilingual training lexicon
    
    Returns
    mapping(np.array): the mapping from English to French vectors
    '''
    x_en, x_fr = [], []
    ## FILL CODE
    for i in range(len(lexicon)):
      x_en.append(word_vectors_en[lexicon[i][0]])
      x_fr.append(word_vectors_fr[lexicon[i][1]])

    x_en =np.array(x_en)
    x_fr =np.array(x_fr)
    M = np.linalg.lstsq(x_en , x_fr)[0]
    return M

In [157]:
mapping = align(word_vectors_en, word_vectors_fr, lexicon)

In [159]:
mapping

array([[-0.06183285, -0.01071552,  0.00175985, ..., -0.01107046,
         0.01629405, -0.01644996],
       [-0.01655313, -0.02930488,  0.09810107, ..., -0.01744702,
        -0.02848298,  0.02070179],
       [-0.01970861, -0.0147154 ,  0.01231819, ...,  0.03036093,
        -0.00209909, -0.00944313],
       ...,
       [ 0.0669847 ,  0.02351181,  0.02041902, ...,  0.00886501,
         0.08635366,  0.00595836],
       [ 0.01936122,  0.00552446,  0.01234669, ..., -0.00623332,
        -0.05116348,  0.05634361],
       [ 0.00530333, -0.03424679, -0.03369923, ..., -0.01344391,
        -0.00051053, -0.00491391]])

Given a mapping, a set of word English word vector and French word vectors, the next function will translate the English word to French. To do so, we apply the mapping on the English word, and retrieve the nearest neighbor of the obtained vector in the set of French word vectors. The translation is then the corresponding French word.

In [ ]:
def translate(word, word_vectors_en, word_vectors_fr, mapping):
    '''
    Parameters:
    word(string): an English word
    word_vectors_en(dict: string -> np.array): English word vectors
    word_vectors_fr(dict: string -> np.array): French word vectors
    mapping(np.array): the mapping from English to French vectors
    
    Returns
    A string containing the translation of the English word
    '''
    ## FILL CODE
    word_vects = word_vectors_en[word]
    np.dot(word_vects , mapping)
    
      
      



    return None

In [ ]:
print(translate("world", word_vectors_en, word_vectors_fr, mapping))
print(translate("machine", word_vectors_en, word_vectors_fr, mapping))
print(translate("learning", word_vectors_en, word_vectors_fr, mapping))

Finally, let's implement a function to evaluate this method on the validation lexicon:

In [ ]:
def evaluate(valid, word_vectors_en, word_vectors_fr, mapping):
    '''
    Parameters:
    valid(a list of pairs of string): the validation lexicon
    word_vectors_en(dict: string -> np.array): English word vectors
    word_vectors_en(dict: string -> np.array): French word vectors
    mapping(np.array): the mapping from English to French vectors
    
    Returns
    Accuracy(float): the accuracy on the validation lexicon
    '''
    acc, n = 0.0, 0
    
    ## FILL CODE

    return acc

In [ ]:
evaluate(valid, word_vectors_en, word_vectors_fr, mapping)